# Import libraries 

In [ ]:
!nvidia-smi

In [2]:
!gmx

                         :-) GROMACS - gmx, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMACS develo

# Set the directory for the simulation

In [ ]:
os.chdir('/home/ali/qbio_Lab2/cg_simulations/new_simu6')

# Prepare the system

In [ ]:
!martinize2 -f model_start_aligned.pdb -ff martini3001 -x start_cg.pdb -o topoltemp.top -dssp /usr/bin/mkdssp -elastic -scfix -cys 0.40 -p backbone -pf 1000

In [ ]:
!python2.7 insane_altered.py -salt 0.15 -charge -0 -sol W -f start_cg.pdb -o system.gro -pbc rectangular -d 30 -x 9 -y 9 -z 12 -l POPC:100 -p topol.top

In [ ]:
!sed -i -e "s/Protein        1/molecule_0 1\nmolecule_1 1/g" ./topol.top

In [ ]:
!cat system.gro

# Energy Minimization

In [ ]:
#######6.0 double précision, pas trop utile#########
!gmx grompp -f step6.0_minimization.mdp -o step6.0_minimization.tpr -c system.gro -r system.gro -p topol.top

In [ ]:
!/usr/local/gromacs-2020.6/bin/gmx_mpi

In [ ]:
!gmx mdrun -deffnm step6.0_minimization -gpu_id 1

In [ ]:
!gmx grompp -f step6.1_minimization.mdp -o step6.1_minimization.tpr -c step6.0_minimization.gro -r step6.0_minimization.gro -p topol.top

In [ ]:
!gmx mdrun -rdd 3.0 -deffnm step6.1_minimization -v -nt 4 -gpu_id 0

# Equilibration Steps - NVT

In [ ]:
!gmx grompp -f step6.2_equilibration.mdp -o step6.2_equilibration.tpr -c step6.1_minimization.gro -r step6.1_minimization.gro -p topol.top

In [ ]:
!gmx mdrun -rdd 3.0 -deffnm step6.2_equilibration -v -nt 4 -gpu_id 0

# Equilibration Steps - NPT

In [ ]:
!gmx grompp -f step6.3_equilibration.mdp -o step6.3_equilibration.tpr -c step6.2_equilibration.gro -r step6.2_equilibration.gro -p topol.top

In [ ]:
!gmx mdrun -rdd 3.0 -deffnm step6.3_equilibration -gpu_id 0 -v -nt 4

In [ ]:
!gmx grompp -f step6.4_equilibration.mdp -o step6.4_equilibration.tpr -c step6.3_equilibration.gro -r step6.3_equilibration.gro -p topol.top

In [ ]:
!gmx mdrun -rdd 3.0 -deffnm step6.4_equilibration -gpu_id 0 -v -nt 4

In [ ]:
!gmx grompp -f step6.5_equilibration.mdp -o step6.5_equilibration.tpr -c step6.4_equilibration.gro -r step6.4_equilibration.gro -p topol.top

In [ ]:
!gmx mdrun -rdd 3.0 -deffnm step6.5_equilibration -gpu_id 0 -v -nt 4

In [ ]:
!gmx grompp -f step6.6_equilibration.mdp -o step6.6_equilibration.tpr -c step6.5_equilibration.gro -r step6.5_equilibration.gro -p topol.top

In [ ]:
!gmx mdrun -rdd 3.0 -deffnm step6.6_equilibration -gpu_id 0 -v -nt 4

# MD run 

In [ ]:
!gmx grompp -f step7_production.mdp -o step7_production.tpr -c step6.6_equilibration.gro -r step6.6_equilibration.gro -p topol.top

In [ ]:
!gmx mdrun -rdd 3.0 -deffnm step7_production -gpu_id 0 -v -nt 4

In [ ]:
!gmx check -f step7_production.xtc

In [ ]:
#!gmx mdrun -rdd 3.0 -deffnm step7_production -cpi step7_production.cpt -gpu_id 0 -v -nt 18

# make an Index

In [ ]:
#get the number of the first and last atoms of the receptor and peptide
!cat system.gro

In [ ]:
!echo -e "a 1-725  \nname 22 rec \na 726-750 \nname 23 pep \nq" |gmx make_ndx -f system.gro -o index.ndx

# a 1-725
# name 22 rec
# a 726-750
# name 23 pep

# Center the the box

In [ ]:
!echo 22 0 | gmx trjconv -f step7_production.xtc -o step7_production.pbc.xtc -s step7_production.tpr -pbc mol -center -n index.ndx 

# NOW READY TO BE ANALYZED!!